In [1]:
import ROOT
import numpy as np 
from matplotlib import pyplot 
import array as array
from array import array
from collections import OrderedDict

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import sys
sys.path.insert(0,'../test/')
from DrawHistogram import *
from Samples_WSel_nom import checkDict, dictSamples
sys.path.insert(0,'../python/')
import CMS_lumi as CMS_lumi

import tdrstyle as tdrstyle
from variables_WSel_fine import nSubVariables_WSel as variables

Welcome to JupyROOT 6.24/06


/tmp/ipykernel_3100/2454220031.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
#Taken from Robin: https://github.com/raggleton/QGAnalysisPlotting/blob/26bb66e690a4a052b9b1acc328059a372fd25c6b/my_unfolder.py#L2358

def renorm(arr2d, axis):
    # create version where each axis summed to 1
    # use where and out args to ensure nans are made into 0s
    summed = arr2d.sum(axis=axis, keepdims=True)
    # If you get "TypeError: No loop matching the specified signature and casting was found for ufunc true_divide"
    # then you should create your arr2d with 'type=float'.
    # I don't understand why: https://github.com/numpy/numpy/issues/10565
    return np.divide(arr2d, summed, where=summed!=0, out=np.zeros_like(arr2d))

def concat_row(arr2d, row_ind):
    # concat row row_ind + row_ind+1
    nrows, ncols = arr2d.shape
    if row_ind > nrows - 2:
        raise IndexError("Cannot concat row [%d] as only %d rows in matrix" % (row_ind, nrows))
    arr2d_new = np.zeros(shape=(nrows-1, ncols), dtype=float)
    new_row = arr2d[row_ind] + arr2d[row_ind+1]
    arr2d_new[row_ind] = new_row
    # fill in new matrix
    if row_ind > 0:
        # do that bit before the new row
        arr2d_new[:row_ind, ] = arr2d[:row_ind, ]
    if row_ind < nrows - 2:
        arr2d_new[row_ind+1:, :] = arr2d[row_ind+2:, :]
    return arr2d_new

def get_th1_bin_centers(h):
    # TODO maintain same shape(1, n) as in th1_to_array?
    centers = np.array([h.GetBinLowEdge(i) + 0.5*h.GetBinWidth(i) for i in range(1, h.GetNbinsX()+1)])
    return centers

def scale_th2_bin_widths(h2d, bins):
    """Scale bins of a square TH2 by bin widths
    bins is a list of bin edges, must have 1 more value than the number of bins in h2d
    """
    if len(bins) != h2d.GetNbinsX()+1:
        print(bins)
        print(h2d.GetNbinsX())
        raise ValueError("Wrong number of bins to scale x axis")
    if len(bins) != h2d.GetNbinsY()+1:
        raise ValueError("Wrong number of bins to scale y axis")
    for ix, (binx_low, binx_high) in enumerate(zip(bins[:-1], bins[1:]), 1):
        for iy, (biny_low, biny_high) in enumerate(zip(bins[:-1], bins[1:]), 1):
            width_x = binx_high - binx_low
            width_y = biny_high - biny_low
            scale = width_x * width_y
            value = h2d.GetBinContent(ix, iy)
            err = h2d.GetBinError(ix, iy)
            h2d.SetBinContent(ix, iy, value / scale)
            h2d.SetBinError(ix, iy, err / scale)
                
def get_th1_bin_widths(h):
    # TODO maintain same shape(1, n) as in th1_to_array?
    widths = np.array([h.GetBinWidth(i) for i in range(1, h.GetNbinsX()+1)])
    return widths
def get_unique_str():
    return ROOT.TUUID().AsString()
def ndarray_to_th1(nd_array, has_oflow_x=False, offset=0.5, bins=None):
    """Convert numpy ndarray row vector to TH1, with shape (1, nbins)
    Use has_oflow_x to include the under/overflow bins
    """
    nbinsx = nd_array.shape[1]
    nbins_hist = nbinsx
    if has_oflow_x:
        nbins_hist -= 2

    # need the 0.5 offset to match TUnfold
    if bins == None:
        h = ROOT.TH1F(get_unique_str(), "", nbins_hist, offset, nbins_hist+offset)
    else:
        if len(bins) != nbinsx+1:
            raise IndexError("len(bins) != nbinsx + 1")
        h = ROOT.TH1F(get_unique_str(), "", nbins_hist, bins)

    x_start = 1
    x_end = nbins_hist

    if has_oflow_x:
        x_start = 0
        x_end = nbins_hist+1

    for x_ind, ix in enumerate(range(x_start, x_end+1)):
        h.SetBinContent(ix, nd_array[0][x_ind])
        h.SetBinError(ix, math.sqrt(abs(nd_array[0][x_ind])))
        #FIXME how to do errors
    return h

def th1_to_ndarray(hist_A, oflow_x=False):
    """Convert TH1 to numpy ndarray"""
    ncol = hist_A.GetNbinsX()
    if oflow_x:
        ncol += 2

    # makes column vectors
    contents = np.zeros(shape=(1, ncol), dtype=np.float64)
    errors = np.zeros(shape=(1, ncol), dtype=np.float64)

    # Get ROOT indices to loop over
    x_start = 0 if oflow_x else 1
    x_end = hist_A.GetNbinsX()
    if oflow_x:
        x_end += 1

    # x_ind for numpy as always starts at 0
    # ix for ROOT
    for x_ind, ix in enumerate(range(x_start, x_end+1)):
        contents[0][x_ind] = hist_A.GetBinContent(ix)
        errors[0][x_ind] = hist_A.GetBinError(ix)

    # check sparsity
    return contents, errors

def th2_to_tmatrixd(hist, include_uflow=False, include_oflow=False):
    n_rows = hist.GetNbinsY()
    n_cols = hist.GetNbinsX()

    # ignore for now as too complicated
    # if include_uflow:
    #     n_rows += 1
    #     n_cols += 1
    # if include_oflow:
    #     n_rows += 1
    #     n_cols += 1

    # taken from https://root.cern.ch/doc/master/TH2_8cxx_source.html#l03739
    m = ROOT.TMatrixD(n_rows, n_cols)
    ilow = m.GetRowLwb()
    iup  = m.GetRowUpb()
    jlow = m.GetColLwb()
    jup  = m.GetColUpb()
    for i in range(ilow, iup+1):
        for j in range(jlow, jup+1):
            m[i,j] = hist.GetBinContent(j-jlow+1,i-ilow+1)
    return m

def th2_to_ndarray(hist_A, oflow_x=False, oflow_y=False):
    """Convert TH2 to numpy ndarray
    Don't use verison in common_utils - wrong axes?
    """
    ncol = hist_A.GetNbinsX()
    if oflow_x:
        ncol += 2
    nrow = hist_A.GetNbinsY()
    if oflow_y:
        nrow += 2

    contents = np.zeros(shape=(nrow, ncol), dtype=np.float64)
    errors = np.zeros(shape=(nrow, ncol), dtype=np.float64)
    # access via contents[irow][icol]

    # Get ROOT indices to loop over
    y_start = 0 if oflow_y else 1
    y_end = hist_A.GetNbinsY()
    if oflow_y:
        y_end += 1

    x_start = 0 if oflow_x else 1
    x_end = hist_A.GetNbinsX()
    if oflow_x:
        x_end += 1

    # y_ind, x_ind for numpy as always starts at 0
    # iy, ix for ROOT
    for y_ind, iy in enumerate(range(y_start, y_end+1)):
        for x_ind, ix in enumerate(range(x_start, x_end+1)):
            contents[y_ind][x_ind] = hist_A.GetBinContent(ix, iy)
            errors[y_ind][x_ind] = hist_A.GetBinError(ix, iy)

    return contents, errors


def ndarray_to_th2(data, offset=0, binsx=None, binsy=None):
    nbinsy, nbinsx = data.shape
    bins_x = array('d', [x+offset for x in range(1, nbinsx+2)]) # e.g. offset = -0.5 for TUnfold
    bins_y = array('d', [x+offset for x in range(1, nbinsy+2)])
    if binsx is not None:
        if len(binsx) != nbinsx+1:
            raise IndexError("binsx wrong size")
        bins_x = binsx
    if binsy is not None:
        if len(binsy) != nbinsy+1:
            raise IndexError("binsy wrong size")
        bins_y = binsy
    h = ROOT.TH2D(get_unique_str(), "", nbinsx, bins_x, nbinsy, bins_y)
    for ix in range(nbinsx):
        for iy in range(nbinsy):
            h.SetBinContent(ix+1, iy+1, data[iy,ix])
            h.SetBinError(ix+1, iy+1, 0)
    return h

def make_hist_from_diagonal_errors(h2d, do_sqrt=True, set_errors=True, offset=0.):
    """Make 1D hist, with errors or contents set to diagonal elements from h2d
    Can be TH2 or numpy.ndarray, cos we have to use both
    Yes that is majorly wack
    set_errors: True to set TH1 bin errors, otherwise sets bin contents
    offset is on bin edge from 0 (TUnfold is 0.5)
    """
    if isinstance(h2d, ROOT.TH2):
        nbins = h2d.GetNbinsX()
        hnew = ROOT.TH1D("h_diag" + get_unique_str(), "", nbins, offset, nbins+offset)
        for i in range(1, nbins+1):
            err = h2d.GetBinContent(i, i)
            if do_sqrt and err > 0:
                err = math.sqrt(err)
            if set_errors:
                hnew.SetBinContent(i, 0)
                hnew.SetBinError(i, err)
            else:
                hnew.SetBinContent(i, err)
                hnew.SetBinError(i, 0)
        return hnew
    elif isinstance(h2d, np.ndarray):
        nbins = h2d.shape[0]
        hnew = ROOT.TH1D("h_diag" + get_unique_str(), "", nbins, offset, nbins+offset)
        for i in range(1, nbins+1):
            err = h2d[i-1, i-1]
            if do_sqrt and err > 0:
                err = math.sqrt(err)
            if set_errors:
                hnew.SetBinContent(i, 0)
                hnew.SetBinError(i, err)
            else:
                hnew.SetBinContent(i, err)
                hnew.SetBinError(i, 0)
        return hnew

def update_hist_bin_error(h_orig, h_to_be_updated):
    """Change the errors in h_to_be_updated to those from h_orig"""
    if h_orig.GetNbinsX() != h_to_be_updated.GetNbinsX():
        raise RuntimeError("Need same # x bins, %d vs %s" % (h_orig.GetNbinsX(), h_to_be_updated.GetNbinsX()))
    for i in range(0, h_orig.GetNbinsX()+2):
        h_to_be_updated.SetBinError(i, h_orig.GetBinError(i))

def get_folded_unfolded(folded, unfolded, cov_tot, probaM):
    # don't use getfoldedoutput, because it doesn't have the updated errors from the total error matrix
    # so we'll have to do it ourselves
    # 1. Make unfolded hist into TVector/TMatrix

    # 2. Make response 2d hist into matrix

    # 3. Multiply the two, convert to TH1

    # Get the TUnfold one for reference, although its errors will be wrong
    
    probaM, _ = th2_to_ndarray(probaM)
    
    folded_unfolded_tunfold = folded

    oflow = False

    # Get unfolded results as array
    unfolded_vector, _ = th1_to_ndarray(unfolded, oflow)

    # Multiply
    # Note that we need to transpose from row vec to column vec
    folded_vec = probaM.dot(unfolded_vector.T)

    # Convert vector to TH1
    folded_unfolded = ndarray_to_th1(folded_vec.T, has_oflow_x=oflow, offset=0.)

    # Error propagation: if y = Ax, with covariance matrices Vyy and Vxx,
    # respectively, then Vyy = (A*Vxx)*A^T
    unfolded_covariance_matrix, _ = th2_to_ndarray(cov_tot , oflow_x=oflow, oflow_y=oflow)
    result = probaM.dot(unfolded_covariance_matrix)
    folded_covariance = result.dot(probaM.T)
    folded_errors = make_hist_from_diagonal_errors(folded_covariance)
    update_hist_bin_error(h_orig=folded_errors, h_to_be_updated=folded_unfolded)

    return folded_unfolded

In [3]:
def th2_to_np_arr(h):
    array = np.zeros((h.GetNbinsY(), h.GetNbinsX()), dtype=float)
    errors = np.zeros((h.GetNbinsY(), h.GetNbinsX()), dtype=float)
    for ix in range(1, h.GetNbinsX() + 1):
        for iy in range(1, h.GetNbinsY() + 1):
            array[iy-1][ix-1] = h.GetBinContent(ix, iy)
            errors[iy-1][ix-1] = h.GetBinError(ix, iy)
    return array, errors

def get_genORreco_bins_from_resp(resp, axis='gen'):
    return resp.sum(axis=0 if axis.lower() == 'gen' else 1)

In [4]:
%jsroot on

In [5]:
def rebinTH2VarBinWidth(ath2, newGenBin, newRecoBin, withUFL=True):
    
    if withUFL: z=0
    else: z=1
    h = ROOT.TH2F( ath2.GetName()+"_Rebin", ath2.GetName()+"_Rebin", len(newGenBin)-1, array( 'd', newGenBin), len(newRecoBin)-1, array( 'd', newRecoBin) )
    h.Sumw2()
    xaxis = ath2.GetXaxis(); 
    yaxis = ath2.GetYaxis(); 

    for j in range(z, yaxis.GetNbins()+1):
        for i in range(z, xaxis.GetNbins()+1): 
            h.Fill(xaxis.GetBinCenter(i), yaxis.GetBinCenter(j),ath2.GetBinContent(i,j))
    return h


### Optimize binnings over metrics of purity and stability

### Trying here with acceptance factors, reverting to my old p/s calculation code

In [6]:
def purity_stability(h_resp, h_gen, h_reco):
    
    hx=h_gen.Clone()
    hy=h_reco.Clone()    
    
    nreco_pergenbin=[]
    nreco_genanywhere=[]
    ngen_recoanywhere=[]
    
    for i in range(0,hx.GetNbinsX()+1):
        nreco_pergenbin.append(h_resp.GetBinContent(i,i))
        nreco_genanywhere.append(hy.GetBinContent(i))
        ngen_recoanywhere.append(hx.GetBinContent(i))
    
    nreco_pergenbin=np.array(nreco_pergenbin)
    nreco_genanywhere=np.array(nreco_genanywhere)
    ngen_recoanywhere=np.array(ngen_recoanywhere)
    
    purity_arr=1.0*nreco_pergenbin/nreco_genanywhere
    stability_arr=1.0*nreco_pergenbin/ngen_recoanywhere
    
    for i in range(0,purity_arr.shape[0]):
        if np.isinf(purity_arr[i])==True or np.isnan(purity_arr[i])==True: purity_arr[i]=0.
        if np.isinf(stability_arr[i])==True or np.isnan(stability_arr[i])==True: stability_arr[i]=0.
    
    return purity_arr,stability_arr

def rebinTH2VarBinWidth(ath2, newGenBin, newRecoBin, withUFL=True):
    
    if withUFL: z=0
    else: z=1
    h = ROOT.TH2F( ath2.GetName()+"_Rebin", ath2.GetName()+"_Rebin", len(newGenBin)-1, array( 'd', newGenBin), len(newRecoBin)-1, array( 'd', newRecoBin) )
    h.Sumw2()
    xaxis = ath2.GetXaxis(); 
    yaxis = ath2.GetYaxis(); 

    for j in range(z, yaxis.GetNbins()+1):
        for i in range(z, xaxis.GetNbins()+1): 
            h.Fill(xaxis.GetBinCenter(i), yaxis.GetBinCenter(j),ath2.GetBinContent(i,j))
    return h

### Nominal

In [11]:
#modified to ensure stability<=1 as per robin's code
for sel in ['WSel', 'topSel']:
    tauBins_WSel = OrderedDict()
    tauBins_topSel = OrderedDict()

    f = ROOT.TFile.Open('../test/Samples/RunIISummer20UL/TTToSemileptonic_powheg_pythia8_2017UL_%s_nom.root'%sel,'READ')
    t = f.Get('jetObservables')

    
    for var in variables:
        exp_purity = 0.4 if var.endswith(('_tau_0p5_4','_tau_1_4','_tau_2_4','_tau_2_3','_tau_1_3', '_tau_0p5_3', )) else 0.65
        exp_stability = 0.5 if var.endswith(('_tau_0p5_4','_tau_1_4','_tau_2_4','_tau_2_3','_tau_1_3', '_tau_0p5_3',)) else 0.55        
        print (var)

        # get response matrix andd accepgen histo to initialize problem
        h_resp = t.Get('resp'+var+'_nom_%s'%sel)
        resp_array2d, _ = th2_to_np_arr(h_resp)
        resp_array2d_copy = np.copy(resp_array2d)
        
        # to get old reco bin edges
        axis = h_resp.GetYaxis() 
        old_gen_bins = [axis.GetBinLowEdge(i) for i  in range(1,axis.GetNbins()+2)]
        new_gen_bins = np.array(old_gen_bins[:])
        
        ibin = 0 
        c = 0
        
        #setup copies that you can start the process from, rebin copy to be able to retain acceptance info required for correcting RM ProjX()
        h_resp_copy = h_resp.Clone()
        #h_accepgen_copy = h_accep.Clone()
        
        while ibin < len(resp_array2d_copy)-1 and c<100000:
            c+=1
            
            resp_array2d_normX = renorm(resp_array2d_copy, axis=0) #should we not include UFL here?
            resp_array2d_normY = renorm(resp_array2d_copy, axis=1)
            purity = resp_array2d_normY[ibin][ibin]
            stability = resp_array2d_normX[ibin][ibin]
            

            if c%100==0: print (c, ibin, 'still running on %s'%var, purity, stability,)

            if purity>exp_purity and stability>exp_stability:

                print("ibin:", ibin, " = ", new_gen_bins[ibin], "-", new_gen_bins[ibin+1], "purity: %.3f" % purity, "stability: %.3f" % stability, c)
                ibin+=1
                continue
            else:
                #concatenate a row and column corresponding to ibin to push up bin edge
                resp_array2d_copy = concat_row(resp_array2d_copy, ibin) # row
                resp_array2d_copy = concat_row(resp_array2d_copy.T, ibin).T # do transposes to apply same as above to columns
                #remove concatenated bin
                new_gen_bins = np.delete(new_gen_bins, ibin+1)

                #h_accepgen.Rebin(len(new_gen_bins)-1, "accepted_gen", new_gen_bins)
                #h_truereco.Rebin(len(new_gen_bins)-1, "true_reco", new_gen_bins)
                #h_resp_copy = rebinTH2VarBinWidth(h_resp_copy, new_gen_bins, new_gen_bins, withUFL=True)
                #h_accepgen_copy = h_accepgen_copy.Rebin(len(new_gen_bins)-1, "accepted_gen", new_gen_bins)
                #if c%100==0: print (f"h_resp bins: {h_resp_copy.GetNbinsX(), h_resp_copy.GetNbinsY()}, h_accepgen bins:{h_accepgen_copy.GetNbinsX()}")

                continue

        print("ibin:", ibin, " = ", new_gen_bins[ibin], "-", new_gen_bins[ibin+1], "purity: %.3f" % purity, "stability: %.3f" % stability, c)

        if purity<exp_purity-0.1 or stability<exp_stability-0.1: 
            new_gen_bins = np.delete(new_gen_bins,-2)
        
        
        print (var, sel, new_gen_bins)

    f.Close()

Jet_tau_0p5_1
100 0 still running on Jet_tau_0p5_1 0.0 0.0
200 0 still running on Jet_tau_0p5_1 1.0 0.03982103619851529
300 0 still running on Jet_tau_0p5_1 0.8592687093007474 0.4506265583167554
ibin: 0  =  0.0 - 0.348 purity: 0.834 stability: 0.552 348
400 1 still running on Jet_tau_0p5_1 0.6127281071327109 0.5255406125082487
ibin: 1  =  0.348 - 0.407 purity: 0.651 stability: 0.567 407
ibin: 2  =  0.407 - 0.465 purity: 0.653 stability: 0.628 465
500 3 still running on Jet_tau_0p5_1 0.5176044963328478 0.5105229326600224
ibin: 3  =  0.465 - 0.518 purity: 0.654 stability: 0.649 518
ibin: 4  =  0.518 - 0.566 purity: 0.653 stability: 0.665 566
600 5 still running on Jet_tau_0p5_1 0.5806899406034626 0.6031277433399271
ibin: 5  =  0.566 - 0.609 purity: 0.652 stability: 0.678 609
ibin: 6  =  0.609 - 0.647 purity: 0.650 stability: 0.671 647
ibin: 7  =  0.647 - 0.682 purity: 0.657 stability: 0.674 682
700 8 still running on Jet_tau_0p5_1 0.4801454744560088 0.49327274716076813
800 8 still runnin

200 0 still running on Jet_tau_0p5_4 0.9181119772981533 0.49068530796936255
ibin: 0  =  0.0 - 0.202 purity: 0.922 stability: 0.501 202
ibin: 1  =  0.202 - 0.253 purity: 0.455 stability: 0.507 253
300 2 still running on Jet_tau_0p5_4 0.3000169464247835 0.5511884273098683
ibin: 2  =  0.253 - 0.328 purity: 0.402 stability: 0.772 328
400 3 still running on Jet_tau_0p5_4 0.3084255912962621 0.7614901130601397
500 3 still running on Jet_tau_0p5_4 0.33034000042340805 0.8142448321731192
600 3 still running on Jet_tau_0p5_4 0.33034000042340805 0.8142448321731192
700 3 still running on Jet_tau_0p5_4 0.33034000042340805 0.8142448321731192
ibin: 3  =  0.328 - 0.799 purity: 0.330 stability: 0.814 798
Jet_tau_0p5_4 WSel [0.    0.202 0.253 0.328 0.799]
Jet_tau_1_4
ibin: 0  =  0.0 - 0.059 purity: 0.944 stability: 0.510 59
ibin: 1  =  0.059 - 0.085 purity: 0.408 stability: 0.501 85
100 2 still running on Jet_tau_1_4 0.17079011691414298 0.3146901184907572
ibin: 2  =  0.085 - 0.135 purity: 0.403 stability

ibin: 4  =  0.152 - 0.193 purity: 0.653 stability: 0.640 193
200 5 still running on Jet_tau_1_2 0.14511107363120088 0.14649702185161606
ibin: 5  =  0.193 - 0.238 purity: 0.653 stability: 0.684 238
ibin: 6  =  0.238 - 0.299 purity: 0.651 stability: 0.787 299
300 7 still running on Jet_tau_1_2 0.01865202247861492 0.026852609828240896
400 7 still running on Jet_tau_1_2 0.5049549819795259 0.7443488152011343
500 7 still running on Jet_tau_1_2 0.5087851955699801 0.7480425322851101
600 7 still running on Jet_tau_1_2 0.5087851955699801 0.7480425322851101
ibin: 7  =  0.299 - 0.6990000000000001 purity: 0.509 stability: 0.748 698
Jet_tau_1_2 topSel [0.    0.011 0.07  0.111 0.152 0.193 0.238 0.699]
Jet_tau_2_2
ibin: 0  =  0.0 - 0.018 purity: 0.917 stability: 0.557 18
ibin: 1  =  0.018 - 0.034 purity: 0.667 stability: 0.623 34
ibin: 2  =  0.034 - 0.053 purity: 0.658 stability: 0.644 53
ibin: 3  =  0.053 - 0.075 purity: 0.651 stability: 0.668 75
100 4 still running on Jet_tau_2_2 0.6182866067797936 

### Nominal w/ 2 btags

In [16]:
for sel in ['WSel', 'topSel']:
    tauBins_WSel = OrderedDict()
    tauBins_topSel = OrderedDict()

    f = ROOT.TFile.Open('../test/Samples/RunIISummer20UL/TTToSemileptonic_powheg_pythia8_2017UL_%s_nom_2btag.root'%sel,'READ')
    t = f.Get('jetObservables')

    
    for var in variables:
        exp_purity = 0.4 if var.endswith(('_tau_0p5_4','_tau_1_4','_tau_2_4','_tau_2_3','_tau_1_3', '_tau_0p5_3', )) else 0.55
        exp_stability = 0.5 if var.endswith(('_tau_0p5_4','_tau_1_4','_tau_2_4','_tau_2_3','_tau_1_3', '_tau_0p5_3',)) else 0.55        
        print (var)

        # get response matrix andd accepgen histo to initialize problem
        h_resp = t.Get('resp'+var+'_nom_%s'%sel)
        resp_array2d, _ = th2_to_np_arr(h_resp)
        resp_array2d_copy = np.copy(resp_array2d)
        
        # to get old reco bin edges
        axis = h_resp.GetYaxis() 
        old_gen_bins = [axis.GetBinLowEdge(i) for i  in range(1,axis.GetNbins()+2)]
        new_gen_bins = np.array(old_gen_bins[:])
        
        ibin = 0 
        c = 0
        
        #setup copies that you can start the process from, rebin copy to be able to retain acceptance info required for correcting RM ProjX()
        h_resp_copy = h_resp.Clone()
        h_accepgen_copy = h_accep.Clone()
        
        while ibin < len(resp_array2d_copy)-1 and c<10000:
            c+=1
            
            resp_array2d_normX = renorm(resp_array2d_copy, axis=0) #should we not include UFL here?
            resp_array2d_normY = renorm(resp_array2d_copy, axis=1)
            purity = resp_array2d_normY[ibin][ibin]
            stability = resp_array2d_normX[ibin][ibin]
            

            if c%100==0: print (c, ibin, 'still running on %s'%var, purity, stability,)

            if purity>exp_purity and stability>exp_stability:

                print("ibin:", ibin, " = ", new_gen_bins[ibin], "-", new_gen_bins[ibin+1], "purity: %.3f" % purity, "stability: %.3f" % stability, c)
                ibin+=1
                continue
            else:
                #concatenate a row and column corresponding to ibin to push up bin edge
                resp_array2d_copy = concat_row(resp_array2d_copy, ibin) # row
                resp_array2d_copy = concat_row(resp_array2d_copy.T, ibin).T # do transposes to apply same as above to columns
                #remove concatenated bin
                new_gen_bins = np.delete(new_gen_bins, ibin+1)

                #h_accepgen.Rebin(len(new_gen_bins)-1, "accepted_gen", new_gen_bins)
                #h_truereco.Rebin(len(new_gen_bins)-1, "true_reco", new_gen_bins)
                #h_resp_copy = rebinTH2VarBinWidth(h_resp_copy, new_gen_bins, new_gen_bins, withUFL=True)
                #h_accepgen_copy = h_accepgen_copy.Rebin(len(new_gen_bins)-1, "accepted_gen", new_gen_bins)
                #if c%100==0: print (f"h_resp bins: {h_resp_copy.GetNbinsX(), h_resp_copy.GetNbinsY()}, h_accepgen bins:{h_accepgen_copy.GetNbinsX()}")

                continue

        if purity<exp_purity or stability<exp_stability: new_gen_bins = np.delete(new_gen_bins,-2)

        print (var, sel, new_gen_bins)

    f.Close()

Jet_tau_0p5_1
100 0 still running on Jet_tau_0p5_1 0.0 0.0
200 0 still running on Jet_tau_0p5_1 0.0 0.0
300 0 still running on Jet_tau_0p5_1 0.9127750497722494 0.48170349897055
ibin: 0  =  0.0 - 0.316 purity: 0.894 stability: 0.552 316
ibin: 1  =  0.316 - 0.382 purity: 0.686 stability: 0.550 382
400 2 still running on Jet_tau_0p5_1 0.24941394926333707 0.24056033060826312
ibin: 2  =  0.382 - 0.432 purity: 0.579 stability: 0.559 432
ibin: 3  =  0.432 - 0.476 purity: 0.575 stability: 0.554 476
500 4 still running on Jet_tau_0p5_1 0.4027958908247876 0.39792505543695933
ibin: 4  =  0.476 - 0.514 purity: 0.556 stability: 0.552 514
ibin: 5  =  0.514 - 0.548 purity: 0.551 stability: 0.551 548
ibin: 6  =  0.548 - 0.58 purity: 0.553 stability: 0.564 580
600 7 still running on Jet_tau_0p5_1 0.427926536611808 0.44364413456159196
ibin: 7  =  0.58 - 0.61 purity: 0.559 stability: 0.579 610
ibin: 8  =  0.61 - 0.637 purity: 0.552 stability: 0.562 637
ibin: 9  =  0.637 - 0.663 purity: 0.558 stability: 0

ibin: 0  =  0.0 - 0.059 purity: 0.743 stability: 1.000 59
100 1 still running on Jet_tau_0p5_4 0.34022673218838406 0.1329120593329102
200 1 still running on Jet_tau_0p5_4 0.9184175659537754 0.4903648039150333
ibin: 1  =  0.059 - 0.202 purity: 0.923 stability: 0.501 202
ibin: 2  =  0.202 - 0.252 purity: 0.439 stability: 0.500 252
300 3 still running on Jet_tau_0p5_4 0.2930137322251483 0.5598612075108231
ibin: 3  =  0.252 - 0.334 purity: 0.403 stability: 0.802 334
400 4 still running on Jet_tau_0p5_4 0.28343184790594883 0.7196632023043233
500 4 still running on Jet_tau_0p5_4 0.30680572875883644 0.7795810666031461
600 4 still running on Jet_tau_0p5_4 0.30680572875883644 0.7795810666031461
700 4 still running on Jet_tau_0p5_4 0.30680572875883644 0.7795810666031461
Jet_tau_0p5_4 WSel [0.    0.059 0.202 0.252 0.799]
Jet_tau_1_4
ibin: 0  =  0.0 - 0.059 purity: 0.943 stability: 0.505 59
ibin: 1  =  0.059 - 0.086 purity: 0.409 stability: 0.518 86
100 2 still running on Jet_tau_1_4 0.14927591200

ibin: 0  =  0.0 - 0.011 purity: 1.000 stability: 1.000 11
ibin: 1  =  0.011 - 0.072 purity: 0.934 stability: 0.556 72
100 2 still running on Jet_tau_1_2 0.5061385026654885 0.4842923768189744
ibin: 2  =  0.072 - 0.106 purity: 0.575 stability: 0.555 106
ibin: 3  =  0.106 - 0.14 purity: 0.589 stability: 0.554 140
ibin: 4  =  0.14 - 0.173 purity: 0.587 stability: 0.556 173
200 5 still running on Jet_tau_1_2 0.5022288685159538 0.5030700619695772
ibin: 5  =  0.173 - 0.205 purity: 0.559 stability: 0.562 205
ibin: 6  =  0.205 - 0.238 purity: 0.552 stability: 0.583 238
ibin: 7  =  0.238 - 0.278 purity: 0.555 stability: 0.659 278
300 8 still running on Jet_tau_1_2 0.3310851685019779 0.44006223192364213
ibin: 8  =  0.278 - 0.351 purity: 0.551 stability: 0.778 351
400 9 still running on Jet_tau_1_2 0.38770933245937195 0.4966273517707321
500 9 still running on Jet_tau_1_2 0.4176750129051051 0.534474956026887
600 9 still running on Jet_tau_1_2 0.4176750129051051 0.534474956026887
Jet_tau_1_2 topSel 

### Nominal w/ mtop pT>400, mSD>150

In [28]:
for sel in ['topSel']:
    tauBins_WSel = OrderedDict()
    tauBins_topSel = OrderedDict()

    f = ROOT.TFile.Open('../test/Samples/RunIISummer20UL/TTToSemileptonic_powheg_pythia8_2017UL_%s_nom_mtop150pt400.root'%sel,'READ')
    t = f.Get('jetObservables')

    
    for var in variables:
        exp_purity = 0.4 if var.endswith(('_tau_0p5_4','_tau_1_4','_tau_2_4','_tau_2_3','_tau_1_3', '_tau_0p5_3', )) else 0.55
        exp_stability = 0.5 if var.endswith(('_tau_0p5_4','_tau_1_4','_tau_2_4','_tau_2_3','_tau_1_3', '_tau_0p5_3',)) else 0.55        
        print (var)

        # get response matrix andd accepgen histo to initialize problem
        h_resp = t.Get('resp'+var+'_nom_%s'%sel)
        resp_array2d, _ = th2_to_np_arr(h_resp)
        resp_array2d_copy = np.copy(resp_array2d)
        
        # to get old reco bin edges
        axis = h_resp.GetYaxis() 
        old_gen_bins = [axis.GetBinLowEdge(i) for i  in range(1,axis.GetNbins()+2)]
        new_gen_bins = np.array(old_gen_bins[:])
        
        ibin = 0 
        c = 0
        
        #setup copies that you can start the process from, rebin copy to be able to retain acceptance info required for correcting RM ProjX()
        h_resp_copy = h_resp.Clone()
        h_accepgen_copy = h_accep.Clone()
        
        while ibin < len(resp_array2d_copy)-1 and c<100000:
            c+=1
            
            resp_array2d_normX = renorm(resp_array2d_copy, axis=0) #should we not include UFL here?
            resp_array2d_normY = renorm(resp_array2d_copy, axis=1)
            purity = resp_array2d_normY[ibin][ibin]
            stability = resp_array2d_normX[ibin][ibin]
            

            if c%100==0: print (c, ibin, 'still running on %s'%var, purity, stability,)

            if purity>exp_purity and stability>exp_stability:

                print("ibin:", ibin, " = ", new_gen_bins[ibin], "-", new_gen_bins[ibin+1], "purity: %.3f" % purity, "stability: %.3f" % stability, c)
                ibin+=1
                continue
            else:
                #concatenate a row and column corresponding to ibin to push up bin edge
                resp_array2d_copy = concat_row(resp_array2d_copy, ibin) # row
                resp_array2d_copy = concat_row(resp_array2d_copy.T, ibin).T # do transposes to apply same as above to columns
                #remove concatenated bin
                new_gen_bins = np.delete(new_gen_bins, ibin+1)

                #h_accepgen.Rebin(len(new_gen_bins)-1, "accepted_gen", new_gen_bins)
                #h_truereco.Rebin(len(new_gen_bins)-1, "true_reco", new_gen_bins)
                #h_resp_copy = rebinTH2VarBinWidth(h_resp_copy, new_gen_bins, new_gen_bins, withUFL=True)
                #h_accepgen_copy = h_accepgen_copy.Rebin(len(new_gen_bins)-1, "accepted_gen", new_gen_bins)
                #if c%100==0: print (f"h_resp bins: {h_resp_copy.GetNbinsX(), h_resp_copy.GetNbinsY()}, h_accepgen bins:{h_accepgen_copy.GetNbinsX()}")

                continue

        print("ibin:", ibin, " = ", new_gen_bins[ibin], "-", new_gen_bins[ibin+1], "purity: %.3f" % purity, "stability: %.3f" % stability, c)

        if purity<exp_purity-0.1 or stability<exp_stability-0.1: 
            new_gen_bins = np.delete(new_gen_bins,-2)
        
        
        print (var, sel, new_gen_bins)

    f.Close()

Jet_tau_0p5_1
100 0 still running on Jet_tau_0p5_1 0.0 0.0
200 0 still running on Jet_tau_0p5_1 0.0 0.0
ibin: 0  =  0.0 - 0.249 purity: 0.648 stability: 1.945 249
300 1 still running on Jet_tau_0p5_1 0.6832158049282157 0.42960721469575125
ibin: 1  =  0.249 - 0.335 purity: 0.826 stability: 0.551 335
ibin: 2  =  0.335 - 0.385 purity: 0.657 stability: 0.552 385
400 3 still running on Jet_tau_0p5_1 0.2885679596302427 0.26238373387687375
ibin: 3  =  0.385 - 0.426 purity: 0.616 stability: 0.553 426
ibin: 4  =  0.426 - 0.463 purity: 0.616 stability: 0.556 463
ibin: 5  =  0.463 - 0.495 purity: 0.582 stability: 0.552 495
500 6 still running on Jet_tau_0p5_1 0.13991920891402582 0.12931211920719332
ibin: 6  =  0.495 - 0.524 purity: 0.575 stability: 0.550 524
ibin: 7  =  0.524 - 0.551 purity: 0.564 stability: 0.553 551
ibin: 8  =  0.551 - 0.577 purity: 0.561 stability: 0.562 577
600 9 still running on Jet_tau_0p5_1 0.5364373376655796 0.5411101466818773
ibin: 9  =  0.577 - 0.602 purity: 0.560 stabi

ibin: 0  =  0.0 - 0.017 purity: 0.929 stability: 0.552 17
ibin: 1  =  0.017 - 0.03 purity: 0.642 stability: 0.558 30
ibin: 2  =  0.03 - 0.044 purity: 0.594 stability: 0.561 44
ibin: 3  =  0.044 - 0.059 purity: 0.577 stability: 0.558 59
ibin: 4  =  0.059 - 0.076 purity: 0.559 stability: 0.589 76
ibin: 5  =  0.076 - 0.097 purity: 0.560 stability: 0.649 97
100 6 still running on Jet_tau_2_2 0.09139807879856983 0.12275664843862268
ibin: 6  =  0.097 - 0.131 purity: 0.550 stability: 0.743 131
ibin: 7  =  0.131 - 0.183 purity: 0.552 stability: 0.718 183
200 8 still running on Jet_tau_2_2 0.27573510712870847 0.3217820695976282
300 8 still running on Jet_tau_2_2 0.5163914528399218 0.5930277146642757
400 8 still running on Jet_tau_2_2 0.5163914528399218 0.5930277146642757
ibin: 8  =  0.183 - 0.499 purity: 0.516 stability: 0.593 498
Jet_tau_2_2 topSel [0.    0.017 0.03  0.044 0.059 0.076 0.097 0.131 0.183 0.499]
Jet_tau_0p5_3
ibin: 0  =  0.0 - 0.046 purity: 0.497 stability: 1.000 46
100 1 still r

### Cuts on leptonic jet only (rom)

In [29]:
for sel in ['WSel', 'topSel']:
    tauBins_WSel = OrderedDict()
    tauBins_topSel = OrderedDict()

    f = ROOT.TFile.Open('../test/Samples/RunIISummer20UL/TTToSemileptonic_powheg_pythia8_2017UL_%s_rom.root'%sel,'READ')
    t = f.Get('jetObservables')

    
    for var in variables:
        exp_purity = 0.4 if var.endswith(('_tau_0p5_4','_tau_1_4','_tau_2_4','_tau_2_3','_tau_1_3', '_tau_0p5_3', )) else 0.55
        exp_stability = 0.5 if var.endswith(('_tau_0p5_4','_tau_1_4','_tau_2_4','_tau_2_3','_tau_1_3', '_tau_0p5_3',)) else 0.55        
        print (var)

        # get response matrix andd accepgen histo to initialize problem
        h_resp = t.Get('resp'+var+'_nom_%s'%sel)
        resp_array2d, _ = th2_to_np_arr(h_resp)
        resp_array2d_copy = np.copy(resp_array2d)
        
        # to get old reco bin edges
        axis = h_resp.GetYaxis() 
        old_gen_bins = [axis.GetBinLowEdge(i) for i  in range(1,axis.GetNbins()+2)]
        new_gen_bins = np.array(old_gen_bins[:])
        
        ibin = 0 
        c = 0
        
        #setup copies that you can start the process from, rebin copy to be able to retain acceptance info required for correcting RM ProjX()
        h_resp_copy = h_resp.Clone()
        h_accepgen_copy = h_accep.Clone()
        
        while ibin < len(resp_array2d_copy)-1 and c<100000:
            c+=1
            
            resp_array2d_normX = renorm(resp_array2d_copy, axis=0) #should we not include UFL here?
            resp_array2d_normY = renorm(resp_array2d_copy, axis=1)
            purity = resp_array2d_normY[ibin][ibin]
            stability = resp_array2d_normX[ibin][ibin]
            

            if c%100==0: print (c, ibin, 'still running on %s'%var, purity, stability,)

            if purity>exp_purity and stability>exp_stability:

                print("ibin:", ibin, " = ", new_gen_bins[ibin], "-", new_gen_bins[ibin+1], "purity: %.3f" % purity, "stability: %.3f" % stability, c)
                ibin+=1
                continue
            else:
                #concatenate a row and column corresponding to ibin to push up bin edge
                resp_array2d_copy = concat_row(resp_array2d_copy, ibin) # row
                resp_array2d_copy = concat_row(resp_array2d_copy.T, ibin).T # do transposes to apply same as above to columns
                #remove concatenated bin
                new_gen_bins = np.delete(new_gen_bins, ibin+1)

                #h_accepgen.Rebin(len(new_gen_bins)-1, "accepted_gen", new_gen_bins)
                #h_truereco.Rebin(len(new_gen_bins)-1, "true_reco", new_gen_bins)
                #h_resp_copy = rebinTH2VarBinWidth(h_resp_copy, new_gen_bins, new_gen_bins, withUFL=True)
                #h_accepgen_copy = h_accepgen_copy.Rebin(len(new_gen_bins)-1, "accepted_gen", new_gen_bins)
                #if c%100==0: print (f"h_resp bins: {h_resp_copy.GetNbinsX(), h_resp_copy.GetNbinsY()}, h_accepgen bins:{h_accepgen_copy.GetNbinsX()}")

                continue

        print("ibin:", ibin, " = ", new_gen_bins[ibin], "-", new_gen_bins[ibin+1], "purity: %.3f" % purity, "stability: %.3f" % stability, c)

        if purity<exp_purity-0.1 or stability<exp_stability-0.1: 
            new_gen_bins = np.delete(new_gen_bins,-2)
        
        
        print (var, sel, new_gen_bins)

    f.Close()

Jet_tau_0p5_1
100 0 still running on Jet_tau_0p5_1 0.4151690038304432 0.15170352693624284
200 0 still running on Jet_tau_0p5_1 0.8092648456167474 0.5395866896180386
ibin: 0  =  0.0 - 0.204 purity: 0.812 stability: 0.554 204
ibin: 1  =  0.204 - 0.272 purity: 0.576 stability: 0.551 272
300 2 still running on Jet_tau_0p5_1 0.2958730145751376 0.30753225817315055
ibin: 2  =  0.272 - 0.333 purity: 0.551 stability: 0.555 333
ibin: 3  =  0.333 - 0.389 purity: 0.568 stability: 0.556 389
400 4 still running on Jet_tau_0p5_1 0.15256871897463878 0.14803566613041258
ibin: 4  =  0.389 - 0.439 purity: 0.581 stability: 0.556 439
ibin: 5  =  0.439 - 0.482 purity: 0.572 stability: 0.553 482
500 6 still running on Jet_tau_0p5_1 0.3280056035544153 0.3214830346627606
ibin: 6  =  0.482 - 0.519 purity: 0.556 stability: 0.551 519
ibin: 7  =  0.519 - 0.553 purity: 0.558 stability: 0.559 553
ibin: 8  =  0.553 - 0.584 purity: 0.554 stability: 0.570 584
600 9 still running on Jet_tau_0p5_1 0.37951846382876636 0.3

ibin: 0  =  0.0 - 0.064 purity: 0.938 stability: 0.500 64
ibin: 1  =  0.064 - 0.093 purity: 0.421 stability: 0.507 93
100 2 still running on Jet_tau_1_3 0.08050298292317994 0.13220497302667933
ibin: 2  =  0.093 - 0.138 purity: 0.402 stability: 0.713 138
ibin: 3  =  0.138 - 0.193 purity: 0.401 stability: 0.730 193
200 4 still running on Jet_tau_1_3 0.07538910818350868 0.1434929228135341
ibin: 4  =  0.193 - 0.265 purity: 0.400 stability: 0.693 265
300 5 still running on Jet_tau_1_3 0.24770704168626312 0.29524066591538184
400 5 still running on Jet_tau_1_3 0.320401058695975 0.40904257184080345
ibin: 5  =  0.265 - 0.499 purity: 0.320 stability: 0.409 498
Jet_tau_1_3 WSel [0.    0.064 0.093 0.138 0.193 0.265 0.499]
Jet_tau_2_3
ibin: 0  =  0.0 - 0.014 purity: 0.870 stability: 0.517 14
ibin: 1  =  0.014 - 0.024 purity: 0.447 stability: 0.536 24
ibin: 2  =  0.024 - 0.041 purity: 0.410 stability: 0.650 41
ibin: 3  =  0.041 - 0.073 purity: 0.402 stability: 0.665 73
100 4 still running on Jet_tau

ibin: 13  =  0.223 - 0.246 purity: 0.553 stability: 0.572 246
ibin: 14  =  0.246 - 0.271 purity: 0.561 stability: 0.605 271
ibin: 15  =  0.271 - 0.296 purity: 0.555 stability: 0.611 296
300 16 still running on Jet_tau_2_1 0.11226342474129487 0.12415738035467803
ibin: 16  =  0.296 - 0.324 purity: 0.555 stability: 0.653 324
ibin: 17  =  0.324 - 0.359 purity: 0.551 stability: 0.682 359
400 18 still running on Jet_tau_2_1 0.5316533039745537 0.6250052506383751
ibin: 18  =  0.359 - 0.401 purity: 0.558 stability: 0.639 401
500 19 still running on Jet_tau_2_1 0.4801168072176696 0.32534676355630654
600 19 still running on Jet_tau_2_1 0.4801168072176696 0.32534676355630654
700 19 still running on Jet_tau_2_1 0.4801168072176696 0.32534676355630654
800 19 still running on Jet_tau_2_1 0.4801168072176696 0.32534676355630654
ibin: 19  =  0.401 - 0.899 purity: 0.480 stability: 0.325 898
Jet_tau_2_1 topSel [0.    0.01  0.016 0.038 0.053 0.069 0.085 0.102 0.12  0.138 0.158 0.179
 0.201 0.223 0.246 0.271

ibin: 0  =  0.0 - 0.012 purity: 0.968 stability: 0.544 12
ibin: 1  =  0.012 - 0.025 purity: 0.407 stability: 0.777 25
100 2 still running on Jet_tau_2_4 0.3505499826407653 0.8772505177117723
200 2 still running on Jet_tau_2_4 0.3505499826407653 0.8772505177117723
300 2 still running on Jet_tau_2_4 0.3505499826407653 0.8772505177117723
400 2 still running on Jet_tau_2_4 0.3505499826407653 0.8772505177117723
ibin: 2  =  0.025 - 0.499 purity: 0.351 stability: 0.877 498
Jet_tau_2_4 topSel [0.    0.012 0.025 0.499]


In [31]:
#Crosscheck on old UL17+18 RunIISummer19UL samples

In [30]:
for sel in ['WSel', 'topSel']:
    tauBins_WSel = OrderedDict()
    tauBins_topSel = OrderedDict()

    f = ROOT.TFile.Open('../test/Samples/July_eta1pt5_Summer19UL_PSChecks/TTToSemileptonic_powheg_pythia8_UL17-18_%s.root'%sel,'READ')
    t = f.Get('jetObservables')

    
    for var in variables:
        exp_purity = 0.4 if var.endswith(('_tau_0p5_4','_tau_1_4','_tau_2_4','_tau_2_3','_tau_1_3', '_tau_0p5_3', )) else 0.55
        exp_stability = 0.5 if var.endswith(('_tau_0p5_4','_tau_1_4','_tau_2_4','_tau_2_3','_tau_1_3', '_tau_0p5_3',)) else 0.55        
        print (var)

        # get response matrix andd accepgen histo to initialize problem
        h_resp = t.Get('resp'+var+'_nom_%s'%sel)
        resp_array2d, _ = th2_to_np_arr(h_resp)
        resp_array2d_copy = np.copy(resp_array2d)
        
        # to get old reco bin edges
        axis = h_resp.GetYaxis() 
        old_gen_bins = [axis.GetBinLowEdge(i) for i  in range(1,axis.GetNbins()+2)]
        new_gen_bins = np.array(old_gen_bins[:])
        
        ibin = 0 
        c = 0
        
        #setup copies that you can start the process from, rebin copy to be able to retain acceptance info required for correcting RM ProjX()
        h_resp_copy = h_resp.Clone()
        h_accepgen_copy = h_accep.Clone()
        
        while ibin < len(resp_array2d_copy)-1 and c<100000:
            c+=1
            
            resp_array2d_normX = renorm(resp_array2d_copy, axis=0) #should we not include UFL here?
            resp_array2d_normY = renorm(resp_array2d_copy, axis=1)
            purity = resp_array2d_normY[ibin][ibin]
            stability = resp_array2d_normX[ibin][ibin]
            

            if c%100==0: print (c, ibin, 'still running on %s'%var, purity, stability,)

            if purity>exp_purity and stability>exp_stability:

                print("ibin:", ibin, " = ", new_gen_bins[ibin], "-", new_gen_bins[ibin+1], "purity: %.3f" % purity, "stability: %.3f" % stability, c)
                ibin+=1
                continue
            else:
                #concatenate a row and column corresponding to ibin to push up bin edge
                resp_array2d_copy = concat_row(resp_array2d_copy, ibin) # row
                resp_array2d_copy = concat_row(resp_array2d_copy.T, ibin).T # do transposes to apply same as above to columns
                #remove concatenated bin
                new_gen_bins = np.delete(new_gen_bins, ibin+1)

                #h_accepgen.Rebin(len(new_gen_bins)-1, "accepted_gen", new_gen_bins)
                #h_truereco.Rebin(len(new_gen_bins)-1, "true_reco", new_gen_bins)
                #h_resp_copy = rebinTH2VarBinWidth(h_resp_copy, new_gen_bins, new_gen_bins, withUFL=True)
                #h_accepgen_copy = h_accepgen_copy.Rebin(len(new_gen_bins)-1, "accepted_gen", new_gen_bins)
                #if c%100==0: print (f"h_resp bins: {h_resp_copy.GetNbinsX(), h_resp_copy.GetNbinsY()}, h_accepgen bins:{h_accepgen_copy.GetNbinsX()}")

                continue

        print("ibin:", ibin, " = ", new_gen_bins[ibin], "-", new_gen_bins[ibin+1], "purity: %.3f" % purity, "stability: %.3f" % stability, c)

        if purity<exp_purity-0.1 or stability<exp_stability-0.1: 
            new_gen_bins = np.delete(new_gen_bins,-2)
        
        
        print (var, sel, new_gen_bins)

    f.Close()

Jet_tau_0p5_1
100 0 still running on Jet_tau_0p5_1 0.0 0.0
200 0 still running on Jet_tau_0p5_1 1.0 0.07618042056627787
300 0 still running on Jet_tau_0p5_1 0.8471438326979764 0.44330690752287055
ibin: 0  =  0.0 - 0.336 purity: 0.845 stability: 0.551 336
ibin: 1  =  0.336 - 0.393 purity: 0.635 stability: 0.554 393
400 2 still running on Jet_tau_0p5_1 0.11338232172523707 0.10684293968419396
ibin: 2  =  0.393 - 0.439 purity: 0.575 stability: 0.550 439
ibin: 3  =  0.439 - 0.481 purity: 0.560 stability: 0.553 481
500 4 still running on Jet_tau_0p5_1 0.3349209300435744 0.3330316414991366
ibin: 4  =  0.481 - 0.519 purity: 0.554 stability: 0.551 519
ibin: 5  =  0.519 - 0.554 purity: 0.556 stability: 0.566 554
ibin: 6  =  0.554 - 0.587 purity: 0.558 stability: 0.575 587
600 7 still running on Jet_tau_0p5_1 0.29951203285995787 0.30547516103618316
ibin: 7  =  0.587 - 0.617 purity: 0.557 stability: 0.573 617
ibin: 8  =  0.617 - 0.645 purity: 0.562 stability: 0.575 645
ibin: 9  =  0.645 - 0.67 pur

ibin: 0  =  0.0 - 0.014 purity: 0.876 stability: 0.532 14
ibin: 1  =  0.014 - 0.023 purity: 0.417 stability: 0.506 23
ibin: 2  =  0.023 - 0.039 purity: 0.406 stability: 0.639 39
ibin: 3  =  0.039 - 0.07 purity: 0.401 stability: 0.654 70
100 4 still running on Jet_tau_2_3 0.3077787208435694 0.4294358297451775
200 4 still running on Jet_tau_2_3 0.32849767168956306 0.4658608855471821
300 4 still running on Jet_tau_2_3 0.32849767168956306 0.4658608855471821
400 4 still running on Jet_tau_2_3 0.32849767168956306 0.4658608855471821
ibin: 4  =  0.07 - 0.499 purity: 0.328 stability: 0.466 498
Jet_tau_2_3 WSel [0.    0.014 0.023 0.039 0.07  0.499]
Jet_tau_0p5_4
100 0 still running on Jet_tau_0p5_4 0.396233012879598 0.16516138699208321
ibin: 0  =  0.0 - 0.199 purity: 0.910 stability: 0.504 199
200 1 still running on Jet_tau_0p5_4 0.011603280340412981 0.009143086346612928
ibin: 1  =  0.199 - 0.249 purity: 0.461 stability: 0.502 249
300 2 still running on Jet_tau_0p5_4 0.3306947201373058 0.5887420

100 0 still running on Jet_tau_0p5_2 0.46164404101668133 0.30494219722643096
200 0 still running on Jet_tau_0p5_2 0.8797119441021506 0.5182138200107698
ibin: 0  =  0.0 - 0.206 purity: 0.888 stability: 0.552 206
ibin: 1  =  0.206 - 0.263 purity: 0.607 stability: 0.551 263
300 2 still running on Jet_tau_0p5_2 0.4552555184770793 0.4421007929328058
ibin: 2  =  0.263 - 0.314 purity: 0.580 stability: 0.552 314
ibin: 3  =  0.314 - 0.362 purity: 0.587 stability: 0.556 362
400 4 still running on Jet_tau_0p5_2 0.532333952562269 0.5177347400897218
ibin: 4  =  0.362 - 0.404 purity: 0.565 stability: 0.552 404
ibin: 5  =  0.404 - 0.445 purity: 0.553 stability: 0.579 445
ibin: 6  =  0.445 - 0.49 purity: 0.556 stability: 0.648 490
500 7 still running on Jet_tau_0p5_2 0.15570543393487316 0.1964910740847383
ibin: 7  =  0.49 - 0.557 purity: 0.552 stability: 0.742 557
600 8 still running on Jet_tau_0p5_2 0.4025062333977321 0.5212522518466532
700 8 still running on Jet_tau_0p5_2 0.4430298222041554 0.577054

### Optimize binnings over metric of at least 5% of events per bin